<a href="https://colab.research.google.com/github/alibagheribardi/Erdos_Project/blob/main/Projec1_Regression_Modification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.datasets import load_iris

def features_prioritization(X, y, features):
    features_scores = []
    knn_reg = KNeighborsRegressor()

    for feature in features:
        X_feature = X[[feature]]  # Ensure 2D
        knn_reg.fit(X_feature, y)
        r2 = knn_reg.score(X_feature, y)
        features_scores.append((feature, r2))

    df_scores = pd.DataFrame(features_scores, columns=['Feature', 'R2_Score']).sort_values(by='R2_Score', ascending=False)
    return df_scores



In [40]:
# Load iris dataset
iris = load_iris()
X_iris = pd.DataFrame(iris.data, columns=iris.feature_names)  # Convert to DataFrame
y_iris = iris.target
iris_features = iris.feature_names

# Run feature prioritization
iris_scores = features_prioritization(X_iris, y_iris, iris_features)
print(iris_scores)


             Feature  R2_Score
2  petal length (cm)    0.9496
3   petal width (cm)    0.9464
0  sepal length (cm)    0.6916
1   sepal width (cm)   -0.1940
